In [14]:
# importing necessary modules
from pymongo import MongoClient as mc
#from pathlib import Path
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
import pandas as pd
import fingerprint_enhancer
from walking import walking, walkonce, checkstable, mergeneighbors
import math

In [15]:
def get_image_paths_and_mv():
    # this funciton only return image paths from "FVC2000" folder
    main_lst = []
    mvs = []
    paths_list = []

    db_client = mc("10.5.18.101")
    coll = db_client.BI.FVC2004
    coll = coll.find({})
    
    for i in coll:
        #if(so_far < count):
        main_lst.append(['../../../19CS91R05/datasets/' + i['path'] , i['mv'] ])
        paths_list.append('../../../19CS91R05/datasets/' + i['path'])
        mvs.append(i['mv'])
    
    return main_lst

In [16]:
def walk_core2(en_image):
	detect_SP = walking(en_image)
	return detect_SP

In [17]:
# Lets check the Minutiae binary pattern

def checkIfPossible(x,y,image_height, image_width):
    if(x<0 or x>=image_height):
        return False
    elif(y<0 or y>=image_width):
        return False
    return True

def GetMinutiaBinaryPattern(minutiae, fingerprint):
    MBP_List = []
    # fingerprint = cv.imread(fingerprint_path, cv.IMREAD_GRAYSCALE)
    fingerprint = fingerprint_enhancer.enhance_Fingerprint(fingerprint)

    image_height, image_width = fingerprint.shape

    for i in minutiae:
        x, y = i[:2]
        window = [[0 for i in range(3)] for j in range(3)]
        if(checkIfPossible(x-1, y+1,image_height,image_width)):
            window[0][0] = fingerprint[x-1][y+1]
        if(checkIfPossible(x, y+1,image_height,image_width)):
            window[0][1] = fingerprint[x][y+1]
        if(checkIfPossible(x+1, y+1,image_height,image_width)):
            window[0][2] = fingerprint[x+1][y+1]
        if(checkIfPossible(x-1, y,image_height,image_width)):
            window[1][0] = fingerprint[x-1][y]
        if(checkIfPossible(x, y,image_height,image_width)):
            window[1][1] = fingerprint[x][y]
        if(checkIfPossible(x+1, y,image_height,image_width)):
            window[1][2] = fingerprint[x+1][y]
        if(checkIfPossible(x-1, y-1,image_height,image_width)):
            window[2][0] = fingerprint[x-1][y-1]
        if(checkIfPossible(x, y-1,image_height,image_width)):
            window[2][1] = fingerprint[x][y-1]
        if(checkIfPossible(x+1, y-1,image_height,image_width)):
            window[2][2] = fingerprint[x+1][y-1]
        
        temp = []
        
        # direction of MBP vector (0,0) -> (0,1) -> (0,2) -> (1,2) -> (2,2) -> (2,1) -> (2,0) -> (1,0)
        # Hard coding

        temp.append(window[0][0]//255)
        temp.append(window[0][1]//255)
        temp.append(window[0][2]//255)
        temp.append(window[1][2]//255)
        temp.append(window[2][2]//255)
        temp.append(window[2][1]//255)
        temp.append(window[2][0]//255)
        temp.append(window[1][0]//255)
        
        
        # for j in range(3):
        #     for k in range(3):
        #         if(k == 1 and j ==1):
        #             continue
        #         else:
        #             temp.append(window[j][k])
        MBP_List.append(temp)
    return MBP_List

In [18]:
def create_data_frame(lst):
    fingerprint_path, minutia = lst
    # global main_df
    img = cv.imread(fingerprint_path,cv.IMREAD_GRAYSCALE)
    en_image = fingerprint_enhancer.enhance_Fingerprint(img)
    # en_image_bw = cv.cvtColor(en_image, cv.COLOR_GRAY2BGR)
    distance = []
    sectors = []
    try:
        SP = walk_core2(en_image)
    except:
        return
    
    if(min(SP['core'].shape) == 0):
        if min(SP['delta'].shape) == 0:
            # NO Singular Point
            return
        else:
            CorePoint = (int(SP['delta'][0,0]), int(SP['delta'][0,1]))
    else:
        CorePoint = (int(SP['core'][0,0]), int(SP['core'][0,1]))

    for i in minutia:

        dis = ((CorePoint[0] - i[0])**2 + (CorePoint[1] - i[1])**2 )**0.5

        distance.append(dis)

        rads = math.atan2((i[1]- CorePoint[1]), (i[0]- CorePoint[0]))
        
        #if CorePoint[0] == i[0]:
        #    rads = 90
        #else:
        #    rads = math.atan((CorePoint[1] - i[1])/(CorePoint[0] - i[0]))


        degs = math.degrees(rads)
        if degs < 0:
            degs = 360 + degs
        
        sectors.append(int(degs//5 + 1 ))
    
    mbps = GetMinutiaBinaryPattern(minutia, en_image)
    pths = [fingerprint_path for i in range(len(minutia))]
    
    data = {
        "Distance" : distance,
        "Sector" : sectors,
        "MBP" : mbps,
        "Image Path" : pths 
    }
    
    new_df = pd.DataFrame(data)
    
    return new_df



In [19]:
from multiprocessing import Pool
main_df = pd.DataFrame(columns= ['Distance', 'Sector', 'MBP', 'Image Path'])
lst = get_image_paths_and_mv()
with Pool(10) as P:
    ans = P.map(create_data_frame, lst)

In [20]:
con = pd.concat(ans, ignore_index=True)
con.to_csv('FVC_2004_index_space.csv')
con

,Distance,Sector,MBP,Image Path
0,219.572312,50,"[0, 0, 0, 0, 0, 0, 0, 0]",../../../19CS91R05/datasets/FVC2004/Dbs/DB2_B/...
1,221.822001,60,"[1, 1, 1, 1, 1, 1, 0, 1]",../../../19CS91R05/datasets/FVC2004/Dbs/DB2_B/...
2,208.923431,50,"[1, 1, 1, 1, 1, 1, 0, 1]",../../../19CS91R05/datasets/FVC2004/Dbs/DB2_B/...
3,207.530721,49,"[0, 0, 1, 0, 0, 0, 0, 0]",../../../19CS91R05/datasets/FVC2004/Dbs/DB2_B/...
4,173.233946,55,"[1, 1, 1, 1, 1, 1, 1, 1]",../../../19CS91R05/datasets/FVC2004/Dbs/DB2_B/...
...,...,...,...,...
195962,163.049072,15,"[0, 1, 1, 1, 0, 0, 0, 0]",../../../19CS91R05/datasets/FVC2004/Dbs/DB2_A/...
195963,166.865215,16,"[1, 1, 1, 1, 1, 1, 1, 1]",../../../19CS91R05/datasets/FVC2004/Dbs/DB2_A/...
195964,163.783394,20,"[1, 1, 1, 0, 0, 0, 0, 0]",../../../19CS91R05/datasets/FVC2004/Dbs/DB2_A/...
195965,189.446562,18,"[0, 0, 0, 0, 0, 0, 0, 0]",../../../19CS91R05/datasets/FVC2004/Dbs/DB2_A/...


In [21]:
file_minutia = get_image_paths_and_mv()
file_minutia = file_minutia[0]
df = create_data_frame(file_minutia)

sort_main_df = pd.read_csv('FVC_2004_index_space.csv')

list_of_dfs = []
list_of_rows = []
lambda_hamming = lambda mbp, realmbp : sum([1 if mbp[i]==realmbp[i] else 0 for i in range(8) ])


for idx,row in df.iterrows():
    dis = row['Distance']
    sec = row['Sector']
    MBP = row['MBP']
    def hamming_dis(mbp):
        count = 0
        for i in range(8):
            if(mbp[i] == MBP[i]):
                count += 1
        # return True if count > 3 else False
        return count
    list_of_rows.append([dis, sec, MBP])
    right_rows = sort_main_df[(sort_main_df['Distance'] == dis) ] 
    right_rows['NEW_MBP'] = right_rows['MBP'].apply(hamming_dis, 'columns')
    # right_rows = right_rows[right_rows['NEW_MBP'] > 3]
    # print(right_rows)
    list_of_dfs.append(right_rows)

/tmp/ipykernel_45822/2333592665.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  right_rows['NEW_MBP'] = right_rows['MBP'].apply(hamming_dis, 'columns')
/tmp/ipykernel_45822/2333592665.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  right_rows['NEW_MBP'] = right_rows['MBP'].apply(hamming_dis, 'columns')
/tmp/ipykernel_45822/2333592665.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [22]:
final = pd.concat(list_of_dfs)

In [23]:
print(final)

nmbp = final['NEW_MBP']

        Unnamed: 0    Distance  Sector                       MBP  \
1                1  221.822001      60  [1, 1, 1, 1, 1, 1, 0, 1]   
65055        65055  221.822001      21  [0, 0, 0, 0, 0, 0, 0, 0]   
117544      117544  221.822001      24  [0, 0, 0, 0, 0, 0, 0, 0]   
2                2  208.923431      50  [1, 1, 1, 1, 1, 1, 0, 1]   
101160      101160  208.923431      50  [0, 0, 0, 0, 0, 0, 0, 0]   
...            ...         ...     ...                       ...   
145588      145588  133.843192      29  [1, 1, 0, 0, 0, 0, 0, 1]   
168692      168692  133.843192      47  [1, 1, 0, 0, 0, 1, 1, 1]   
169958      169958  133.843192      13  [0, 0, 0, 0, 0, 0, 0, 0]   
190021      190021  133.843192      65  [0, 0, 1, 1, 0, 0, 0, 0]   
193887      193887  133.843192      11  [0, 1, 1, 1, 1, 1, 1, 1]   

                                               Image Path NEW_MBP  
1       ../../../19CS91R05/datasets/FVC2004/Dbs/DB2_B/...       0  
65055   ../../../19CS91R05/datasets/FVC2004/Dbs

In [24]:
len(sort_main_df)

195967

In [25]:
paths = (final['Image Path'])
print(len(paths))
path_lst = list(paths)
print(len(set(path_lst)))

1434
1079


In [26]:
path_dict = dict()

for i in path_lst:
    if i not in path_dict.keys():
        path_dict[i] = 1
    else:
        path_dict[i] += 1


In [27]:
maxm = -1
for i in path_dict.keys():
    maxm = max(maxm, path_dict[i])
print(maxm)

41


In [28]:
max_lst = []
for i in path_dict.keys():
    if path_dict[i] == maxm:
        max_lst.append(i)

In [29]:
max_lst

['../../../19CS91R05/datasets/FVC2004/Dbs/DB2_B/110_2.tif']

In [30]:
file_minutia[0]

'../../../19CS91R05/datasets/FVC2004/Dbs/DB2_B/110_2.tif'